In [21]:
import pandas as pd
import numpy as np
import time

In [4]:
def reformat_date(df):
    appoint_date = []
    for i in range(len(df)):
        day = df[i]
        if pd.isna(day):
            appoint_date.append(np.nan)  
        elif day == 'REDACTED':
            appoint_date.append(np.nan)  
        elif len(day) == 8:
            if int(day[6:8])+2000 > 2021:
                appoint_date.append('19'+day[6:8]+'-'+day[0:5])
            else: 
                appoint_date.append('20'+day[6:8]+'-'+day[0:5] )
        else:
            mon = day[5:8]
            if mon == 'JAN':
                appoint_date.append(day[0:5]+'01'+day[8:])
            elif mon == 'FEB':
                appoint_date.append(day[0:5]+'02'+day[8:])
            elif mon == 'MAR':
                appoint_date.append(day[0:5]+'03'+day[8:])
            elif mon == 'APR':
                appoint_date.append(day[0:5]+'04'+day[8:])
            elif mon == 'MAY':
                appoint_date.append(day[0:5]+'05'+day[8:])
            elif mon == 'JUN':
                appoint_date.append(day[0:5]+'06'+day[8:])
            elif mon == 'JUL':
                appoint_date.append(day[0:5]+'07'+day[8:])
            elif mon == 'AUG':
                appoint_date.append(day[0:5]+'08'+day[8:])
            elif mon == 'SEP':
                appoint_date.append(day[0:5]+'09'+day[8:])
            elif mon == 'OCT':
                appoint_date.append(day[0:5]+'10'+day[8:])
            elif mon == 'NOV':
                appoint_date.append(day[0:5]+'11'+day[8:])
            elif mon == 'DEC':
                appoint_date.append(day[0:5]+'12'+day[8:])
    return appoint_date

In [8]:
df_trr = pd.read_csv("src/data/trr_trr_refresh.csv")
df_trr['trr_datetime'] = pd.to_datetime(df_trr['trr_datetime'])
df_trr['beat'] = df_trr['beat'].astype(int)
df_trr['officer_appointed_date'] = pd.to_datetime(reformat_date(df_trr['officer_appointed_date']))
df_trr['officer_birth_year'] = df_trr['officer_birth_year'].fillna(0.0).astype(int)
df_trr['officer_birth_year'].replace(0, np.nan,inplace=True)
df_trr['officer_age'] = df_trr['officer_age'].astype(int)
df_trr['officer_on_duty'] = df_trr['officer_on_duty'].astype(bool)
df_trr['officer_injured'] = df_trr['officer_injured'].astype(bool)
df_trr['officer_in_uniform'] = df_trr['officer_in_uniform'].astype(bool)
df_trr['subject_birth_year'] = df_trr['subject_birth_year'].astype(int)
df_trr['subject_age'] = df_trr['subject_age'].astype(int)
df_trr['subject_armed'] = df_trr['subject_armed'].astype(bool)
df_trr['subject_injured'] = df_trr['subject_injured'].astype(bool)
df_trr['subject_alleged_injury'] = df_trr['subject_alleged_injury'].astype(bool)
df_trr['notify_oemc'] = df_trr['notify_oemc'].astype(bool)
df_trr['notify_district_sergeant'] = df_trr['notify_district_sergeant'].astype(bool)
df_trr['notify_op_command'] = df_trr['notify_op_command'].astype(bool)
df_trr['notify_det_division'] = df_trr['notify_det_division'].astype(bool)
df_trr['trr_created'] = pd.to_datetime(df_trr['trr_created'])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
df_weapondischarge = pd.read_csv("src/data/trr_weapondischarge_refresh.csv")
df_weapondischarge['firearm_reloaded'] = df_weapondischarge['firearm_reloaded'].astype(bool)
df_weapondischarge['sight_used'] = df_weapondischarge['sight_used'].astype(bool)

In [10]:
df_trrstatus = pd.read_csv("src/data/trr_trrstatus_refresh.csv")
df_trrstatus['officer_appointed_date'] = pd.to_datetime(reformat_date(df_trrstatus['officer_appointed_date']))
df_trrstatus['officer_birth_year'] = df_trrstatus['officer_birth_year'].fillna(0).astype(int)
df_trrstatus['officer_birth_year'].replace(0, np.nan,inplace=True)
df_trrstatus['status_datetime'] = pd.to_datetime(df_trrstatus['status_datetime'])

In [11]:
def reconciliation_race(df):
    race_map = dict({'AMER IND/ALASKAN NATIVE':'NATIVE AMERICAN/ALASKAN NATIVE',
                'AMER INDIAN / ALASKAN NATIVE':'NATIVE AMERICAN/ALASKAN NATIVE',
                'ASIAN / PACIFIC ISLANDER':'ASIAN/PACIFIC ISLANDER',
                'ASIAN/PACIFIC ISLANDER':'ASIAN/PACIFIC ISLANDER',
                'BLACK':'BLACK',
                'UNKNOWN':'NULL',
                 'UNKNOWN / REFUSED':'NULL',
                 'HISPANIC':'HISPANIC',
                 'WHITE':'WHITE'
                })
    for k in race_map:
        if k[0:2] == 'UN':
            df.replace(k, np.nan,inplace=True)
        else:
            df.replace(k,race_map[k] ,inplace=True)

reconciliation_race(df_trr['subject_race'])

In [12]:
def reconciliation_gender(df):
    race_map = dict({'FEMALE':'F',
                     'MALE':'M'
                })
    for k in race_map:
        df.replace(k,race_map[k] ,inplace=True)
        
reconciliation_gender(df_trr['subject_gender'])

In [13]:
def reconciliation_birth_year(df):
    birth_years = df.to_numpy()
    birth_years[birth_years < 100 ] += 1900
    birth_years[birth_years < 200 ] *= 10
    for i in range(1000,2000,100):
        birth_years[birth_years < i ] += 2000-i   
    df = pd.DataFrame(birth_years)

reconciliation_birth_year(df_trr['subject_birth_year'])

In [14]:
# Indoor/outdoor change specific column.
df_trr.loc[(df_trr.indoor_or_outdoor == 'OUTDOOR'), 'indoor_or_outdoor'] = 'Outdoor'
df_trr.loc[(df_trr.indoor_or_outdoor == 'INDOOR'), 'indoor_or_outdoor'] = 'Indoor'

# Street. 1. change to camel case
df_trr['street'] = df_trr['street'].str.title()

# location. 1. change to camel case 2. change specific items to the correct form.
df_trr['location'] = df_trr['location'].str.title()

df_trr.loc[(df_trr.location == 'Cha Hallway / Stairwell / Elevator'), 'location'] = 'Cha Hallway/Stairwell/Elevator'
df_trr.loc[(df_trr.location == 'Cha Parking Lot / Grounds'), 'location'] = 'Cha Parking Lot/Grounds'
df_trr.loc[(df_trr.location == 'Church / Synagogue / Place Of Worship'), 'location'] = 'Church/Synagogue/Place Of Worship'
df_trr.loc[(df_trr.location == 'College / University - Grounds'), 'location'] = 'College/University Grounds'
df_trr.loc[(df_trr.location == 'Factory / Manufacturing Building'), 'location'] = 'Factory/Manufacturing Building'
df_trr.loc[(df_trr.location == 'Government Building / Property'), 'location'] = 'Government Building/Property'
df_trr.loc[(df_trr.location == 'Highway / Expressway'), 'location'] = 'Highway/Expressway'
df_trr.loc[(df_trr.location == 'Hospital Building / Grounds'), 'location'] = 'Hospital Building/Grounds'
df_trr.loc[(df_trr.location == 'Hotel / Motel'), 'location'] = 'Hotel/Motel'
df_trr.loc[(df_trr.location == 'Movie House / Theater'), 'location'] = 'Movie House/Theater'
df_trr.loc[(df_trr.location == 'Nursing / Retirement Home'), 'location'] = 'Nursing Home/Retirement Home'
df_trr.loc[(df_trr.location == 'Other (Specify)'), 'location'] = 'Other'
df_trr.loc[(df_trr.location == 'Other Railroad Property / Train Depot'), 'location'] = 'Other Railroad Prop / Train Depot'
df_trr.loc[(df_trr.location == 'Parking Lot / Garage (Non Residential)'), 'location'] = 'Parking Lot/Garage(Non.Resid.)'
df_trr.loc[(df_trr.location == 'Other Railroad Property / Train Depot'), 'location'] = 'Other Railroad Prop / Train Depot'
df_trr.loc[(df_trr.location == 'Police Facility / Vehicle Parking Lot'), 'location'] = 'Police Facility/Veh Parking Lot'
df_trr.loc[(df_trr.location == 'Residence - Porch / Hallway'), 'location'] = 'Residence Porch/Hallway'
df_trr.loc[(df_trr.location == 'Residence - Garage'), 'location'] = 'Residence-Garage'
df_trr.loc[(df_trr.location == 'Residence - Yard (Front / Back)'), 'location'] = 'Residential Yard (Front/Back)'
df_trr.loc[(df_trr.location == 'School - Private Building'), 'location'] = 'School, Private, Building'
df_trr.loc[(df_trr.location == 'School - Private Grounds'), 'location'] = 'School, Private, Grounds'
df_trr.loc[(df_trr.location == 'School - Public Building'), 'location'] = 'School, Public, Building'
df_trr.loc[(df_trr.location == 'School - Public Grounds'), 'location'] = 'School, Public, Grounds'
df_trr.loc[(df_trr.location == 'Sports Arena / Stadium'), 'location'] = 'Sports Arena/Stadium'
df_trr.loc[(df_trr.location == 'Tavern / Liquor Store'), 'location'] = 'Tavern/Liquor Store'
df_trr.loc[(df_trr.location == 'Vacant Lot / Land'), 'location'] = 'Vacant Lot/Land'
df_trr.loc[(df_trr.location == 'Vehicle - Other Ride Share Service (Lyft, Uber, Etc.)'), 'location'] = 'Vehicle - Other Ride Service'
df_trr.loc[(df_trr.location == 'Vehicle - Commercial'), 'location'] = 'Vehicle-Commercial'
df_trr.loc[(df_trr.location == 'Cta Parking Lot / Garage / Other Property'), 'location'] = 'Cta Garage / Other Property'
df_trr.loc[(df_trr.location == 'Lakefront / Waterfront / Riverbank'), 'location'] = 'Lakefront/Waterfront/Riverbank'
df_trr.loc[(df_trr.location == 'Medical / Dental Office'), 'location'] = 'Medical/Dental Office'
df_trr.loc[(df_trr.location == 'Airport Parking Lot'), 'location'] = 'Airport/Aircraft'
df_trr.loc[(df_trr.location == 'Airport Terminal Mezzanine - Non-Secure Area'), 'location'] = 'Airport Terminal Lower Level - Non-Secure Area'


# birth year
def age_sanity(row):
    if row < 100:
        return 1900 + row
    if 100 <= row < 1900:
        return 1900 + row % 100
    return row

df_trr.subject_birth_year = df_trr.subject_birth_year.apply(age_sanity)


In [16]:
# Read weapon file.
sub_weapon_refresh = pd.read_csv("src/data/trr_subjectweapon_refresh.csv")
# sub_weapon = pd.read_csv("../../trr_subjectweapon.csv")

# Weapon, change specific items to the correct form.
sub_weapon_refresh.loc[(sub_weapon_refresh.weapon_type == 'CHEMICAL WEAPON') | (sub_weapon_refresh.weapon_type == 'TASER / STUN GUN'), 'weapon_type'] = 'OTHER (SPECIFY)'
sub_weapon_refresh.loc[(sub_weapon_refresh.weapon_type == 'VEHICLE'), 'weapon_type'] = 'VEHICLE - ATTEMPTED TO STRIKE OFFICER WITH VEHICLE'


In [17]:
df_trr["suffix_name"] = "NULL"
def update_last_suffix(df):
    suffix_list = ["I", "II", "III", "IV", "V", "JR", "SR"]
    officer_last_name = []
    suffix_name = []
    for i in df['officer_last_name']:
        target_str = i.upper().split(" ")
        if len(target_str) > 1:
            k = 0
            for j in suffix_list:
                if j == ''.join(e for e in target_str[-1] if e.isalnum()):
                    # print(''.join(e for e in target_str[-1] if e.isalnum()))
                    suffix_name.append(j)
                    k +=1
            if k == 0 :
                suffix_name.append(None)
            officer_last_name.append(' '.join(target_str).title())
        else:
            officer_last_name.append(i.title())
            suffix_name.append(None)
    df['officer_last_name'] = officer_last_name
    df['suffix_name'] = suffix_name
    return df
update_last_suffix(df_trr)

,id,event_number,cr_number,beat,block,direction,street,location,trr_datetime,indoor_or_outdoor,...,officer_age,officer_appointed_date,officer_birth_year,officer_unit_name,officer_unit_detail,trr_created,latitude,longitude,point,suffix_name
0,96630,1.611412e+09,NaN,1124,30XX,West,Jackson Blvd,Street,2016-04-23 18:18:00-05:00,NaN,...,30,2014-02-18,NaN,11,NaN,NaN,41.877650,-87.701265,0101000020E61000007554DA87E1EC55C00F6C4BD356F0...,None
1,97999,1.618612e+09,NaN,723,9XX,West,Marquette Rd,Street,2016-07-04 17:06:00-05:00,Outdoor,...,30,2013-11-25,NaN,7,NaN,NaN,41.772444,-87.647030,0101000020E61000008F215BF168E955C03AFD4575DFE2...,None
2,98796,1.623610e+09,NaN,811,53XX,South,Narragansett Ave,Residence,2016-08-23 16:14:00-05:00,Outdoor,...,51,2002-09-30,NaN,8,NaN,NaN,41.795906,-87.782194,0101000020E61000005DE800770FF255C09919EC3CE0E5...,None
3,99293,1.626900e+09,NaN,2232,100XX,South,Emerald Ave,Sidewalk,2016-09-25 20:00:00-05:00,NaN,...,34,2009-12-16,NaN,22,NaN,NaN,41.712237,-87.642172,0101000020E610000017BB7D5619E955C067C17A922ADB...,None
4,99369,1.627410e+09,NaN,634,XX,West,95Th St,Sidewalk,2016-09-30 17:13:00-05:00,NaN,...,46,2006-02-21,NaN,6,NaN,NaN,41.721990,-87.626064,0101000020E6100000FD4CBD6E11E855C0FF04172B6ADC...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17460,6043642,2.620000e+02,NaN,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,-25,2019-02-19,1995.0,25,NaN,2020-05-26 03:42:00-05:00,41.902789,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...,None
17461,6043651,2.620000e+02,NaN,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,-24,2019-08-16,1996.0,25,NaN,2020-05-26 03:45:00-05:00,41.902789,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...,None
17462,6059341,2.014903e+09,NaN,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,-31,2017-09-18,1989.0,10,NaN,2020-05-28 06:52:00-05:00,41.854055,-87.690811,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...,None
17463,6059467,2.014903e+09,NaN,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,-26,2018-11-27,1994.0,10,NaN,2020-05-28 07:24:00-05:00,41.854055,-87.690811,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...,None


In [18]:
def update_first_name(df):
    officer_first_name = []
    for i in df['officer_last_name']:
        officer_first_name.append(i.title())
    df['officer_first_name'] = officer_first_name
    return df
update_first_name(df_trr)

,id,event_number,cr_number,beat,block,direction,street,location,trr_datetime,indoor_or_outdoor,...,officer_age,officer_appointed_date,officer_birth_year,officer_unit_name,officer_unit_detail,trr_created,latitude,longitude,point,suffix_name
0,96630,1.611412e+09,NaN,1124,30XX,West,Jackson Blvd,Street,2016-04-23 18:18:00-05:00,NaN,...,30,2014-02-18,NaN,11,NaN,NaN,41.877650,-87.701265,0101000020E61000007554DA87E1EC55C00F6C4BD356F0...,None
1,97999,1.618612e+09,NaN,723,9XX,West,Marquette Rd,Street,2016-07-04 17:06:00-05:00,Outdoor,...,30,2013-11-25,NaN,7,NaN,NaN,41.772444,-87.647030,0101000020E61000008F215BF168E955C03AFD4575DFE2...,None
2,98796,1.623610e+09,NaN,811,53XX,South,Narragansett Ave,Residence,2016-08-23 16:14:00-05:00,Outdoor,...,51,2002-09-30,NaN,8,NaN,NaN,41.795906,-87.782194,0101000020E61000005DE800770FF255C09919EC3CE0E5...,None
3,99293,1.626900e+09,NaN,2232,100XX,South,Emerald Ave,Sidewalk,2016-09-25 20:00:00-05:00,NaN,...,34,2009-12-16,NaN,22,NaN,NaN,41.712237,-87.642172,0101000020E610000017BB7D5619E955C067C17A922ADB...,None
4,99369,1.627410e+09,NaN,634,XX,West,95Th St,Sidewalk,2016-09-30 17:13:00-05:00,NaN,...,46,2006-02-21,NaN,6,NaN,NaN,41.721990,-87.626064,0101000020E6100000FD4CBD6E11E855C0FF04172B6ADC...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17460,6043642,2.620000e+02,NaN,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,-25,2019-02-19,1995.0,25,NaN,2020-05-26 03:42:00-05:00,41.902789,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...,None
17461,6043651,2.620000e+02,NaN,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,-24,2019-08-16,1996.0,25,NaN,2020-05-26 03:45:00-05:00,41.902789,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...,None
17462,6059341,2.014903e+09,NaN,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,-31,2017-09-18,1989.0,10,NaN,2020-05-28 06:52:00-05:00,41.854055,-87.690811,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...,None
17463,6059467,2.014903e+09,NaN,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,-26,2018-11-27,1994.0,10,NaN,2020-05-28 07:24:00-05:00,41.854055,-87.690811,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...,None


In [19]:
def update_race(df):
    race_map = dict({'AMER IND/ALASKAN NATIVE':'Native American/Alaskan Native',
                'ASIAN/PACIFIC ISLANDER':'Asian/Pacific',
                'BLACK':'Black',
                'UNKNOWN':'Unknown',
                 'HISPANIC':'Hispanic',
                 'WHITE':'White'
                })
    for k in race_map:
        df.replace(k,race_map[k] ,inplace=True)
    return df
update_race(df_trr)

,id,event_number,cr_number,beat,block,direction,street,location,trr_datetime,indoor_or_outdoor,...,officer_age,officer_appointed_date,officer_birth_year,officer_unit_name,officer_unit_detail,trr_created,latitude,longitude,point,suffix_name
0,96630,1.611412e+09,NaN,1124,30XX,West,Jackson Blvd,Street,2016-04-23 18:18:00-05:00,NaN,...,30,2014-02-18,NaN,11,NaN,NaN,41.877650,-87.701265,0101000020E61000007554DA87E1EC55C00F6C4BD356F0...,None
1,97999,1.618612e+09,NaN,723,9XX,West,Marquette Rd,Street,2016-07-04 17:06:00-05:00,Outdoor,...,30,2013-11-25,NaN,7,NaN,NaN,41.772444,-87.647030,0101000020E61000008F215BF168E955C03AFD4575DFE2...,None
2,98796,1.623610e+09,NaN,811,53XX,South,Narragansett Ave,Residence,2016-08-23 16:14:00-05:00,Outdoor,...,51,2002-09-30,NaN,8,NaN,NaN,41.795906,-87.782194,0101000020E61000005DE800770FF255C09919EC3CE0E5...,None
3,99293,1.626900e+09,NaN,2232,100XX,South,Emerald Ave,Sidewalk,2016-09-25 20:00:00-05:00,NaN,...,34,2009-12-16,NaN,22,NaN,NaN,41.712237,-87.642172,0101000020E610000017BB7D5619E955C067C17A922ADB...,None
4,99369,1.627410e+09,NaN,634,XX,West,95Th St,Sidewalk,2016-09-30 17:13:00-05:00,NaN,...,46,2006-02-21,NaN,6,NaN,NaN,41.721990,-87.626064,0101000020E6100000FD4CBD6E11E855C0FF04172B6ADC...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17460,6043642,2.620000e+02,NaN,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,-25,2019-02-19,1995.0,25,NaN,2020-05-26 03:42:00-05:00,41.902789,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...,None
17461,6043651,2.620000e+02,NaN,2534,12XX,N,Kildare Ave,Residence Porch/Hallway,2020-05-26 00:30:00-05:00,Indoor,...,-24,2019-08-16,1996.0,25,NaN,2020-05-26 03:45:00-05:00,41.902789,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...,None
17462,6059341,2.014903e+09,NaN,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,-31,2017-09-18,1989.0,10,NaN,2020-05-28 06:52:00-05:00,41.854055,-87.690811,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...,None
17463,6059467,2.014903e+09,NaN,1023,26XX,W,21St St,Apartment,2020-05-28 05:20:00-05:00,Indoor,...,-26,2018-11-27,1994.0,10,NaN,2020-05-28 07:24:00-05:00,41.854055,-87.690811,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...,None


In [ ]:
# def fuzzy_match(a):
#     first_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']
#     last_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']
# first_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']
# first_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']
# first_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']
# first_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']
# first_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']
# first_name = a['officer_first_name'] == b['officer_first_name'] or not a['officer_first_name'] or not b['officer_first_name']

#     if :
        
# df_trr.index = df_trr.index.map(lambda x: )


In [35]:
# np.array([1,2,3]) - np.array([2])

array([-1,  0,  1])

In [26]:
df_officer = pd.read_csv("src/data/data_officer.csv")
fields = ['first_name', 'middle_initial', 'last_name',
          'suffix_name', 'birth_year', 'appointed_date', 'gender', 'race']
officer_fields = ['officer_first_name', 'officer_middle_initial', 'officer_last_name', 
          'suffix_name', 'officer_birth_year', 'officer_appointed_date', 'officer_gender', 'officer_race']

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [35]:
s = time.time()
index= []
value = []
for i, officer in df_trr.iterrows():
    trr = np.zeros(len(df_officer))
    for j in range(0,7):
        trr += (pd.isna(df_officer[fields[j]]) | pd.isna(officer[officer_fields[j]])).to_numpy()
        trr -= (pd.isna(df_officer[fields[j]]) & pd.isna(officer[officer_fields[j]])).to_numpy()
        trr += (df_officer[fields[j]] == officer[officer_fields[j]]).to_numpy()
    temp = np.argmax(trr)
    value.append(trr[temp])
    index.append(temp)
    break
e =time.time()

e-s

0.0997769832611084

In [39]:
df_trr.loc[0]

id                                                                      96630
event_number                                                     1611411724.0
cr_number                                                                 NaN
beat                                                                     1124
block                                                                    30XX
direction                                                                West
street                                                           Jackson Blvd
location                                                               Street
trr_datetime                                        2016-04-23 18:18:00-05:00
indoor_or_outdoor                                                         NaN
lighting_condition                                                        NaN
weather_condition                                                         NaN
notify_oemc                                                     

In [40]:
df_officer.loc[index[0]]

id                                                       33769
gender                                                       M
race                                                     Black
appointed_date                                             NaN
rank                                                       NaN
active                                                 Unknown
birth_year                                              1915.0
first_name                                             William
last_name                                                Burke
tags                                                       NaN
middle_initial                                               E
suffix_name                                                 JR
resignation_date                                           NaN
complaint_percentile                                       NaN
middle_initial2                                            NaN
civilian_allegation_percentile                         